# Fine Tuning Large Language Model - Model

In this workshop, you will learn how to fine tune the prompts and the LLMs to enhance and improves its response.

In [1]:
# Import libraries
import torch, time
import pandas as pd
import numpy as np
import evaluate
from datasets import load_dataset
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, Trainer, GenerationConfig, TrainingArguments

from peft import PeftModel, LoraConfig, get_peft_model, TaskType

In [2]:
# TODO: Load and explore the following datasets

dataset_name = "knkarthick/dialogsum"
model_name = "google/flan-t5-small"
model_name = "google/flan-t5-base"


In [3]:
# TODO: Print a record
dataset = load_dataset(dataset_name)
print(dataset)
print(dataset.shape)

DatasetDict({
    train: Dataset({
        features: ['id', 'dialogue', 'summary', 'topic'],
        num_rows: 12460
    })
    validation: Dataset({
        features: ['id', 'dialogue', 'summary', 'topic'],
        num_rows: 500
    })
    test: Dataset({
        features: ['id', 'dialogue', 'summary', 'topic'],
        num_rows: 1500
    })
})
{'train': (12460, 4), 'validation': (500, 4), 'test': (1500, 4)}


## Fine tuning the LLM model

In this workshop we will be turning the <code>google/flan-t5-base</code> model.

In [9]:
# Utility function to dump a model's tunable parameters
def print_trainable_model_params(model):
   trainable_model_params = 0
   all_model_params = 0
   for _, param in model.named_parameters():
      all_model_params += param.numel()
      if param.requires_grad:
         trainable_model_params += param.numel()
   return f"""
   Trainable parameters: {trainable_model_params}
   Total parameters: {all_model_params}
   Percentable of trainable parameters: {100 * trainable_model_params / all_model_params:.2f}%
   """

In [10]:
# TODO: Load model
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [11]:
# TODO: Print number of trainable parameters
print_trainable_model_params(model)

'\n   Trainable parameters: 247577856\n   Total parameters: 247577856\n   Percentable of trainable parameters: 100.00%\n   '

### Preprocess the dialogue dataset

We will train the model to summarize dialogue by creating a dialogue-summary pair for the LLM to process. The dialogue is the training data and the summary is the label. This is supervized learning.

The prompt will be as follows

```
Summarize the following dialogue.\n
\n
Fred: ...\n
Barney: ...\n
\n
Summary:\n
Summary of the conversation between Fred and Barney
```

The prompt and the summary will be tokenized for the LLM

In [12]:
# Utitlity function to prepare the data for training 
# Tokenize function
def tokenize_fn(data):
   start_prompt = 'Summarize the following dialogue.\n\n'
   end_prompt = '\n\nSummary:'
   prompt = [ start_prompt + d + end_prompt for d in data['dialogue'] ]
   summary = data['summary']
   data['input_ids'] = tokenizer(prompt, padding="max_length", truncation=True, return_tensors="pt").input_ids
   data['labels'] = tokenizer(summary, padding="max_length", truncation=True, return_tensors="pt").input_ids
   return data


In [13]:
# TODO: prepare the data for training with the tokenize_fn function
tokenized_dataset = dataset.map(
   tokenize_fn,
   batched=True
)

In [18]:
# TODO: Verify prepared data
idx = 500 
for k, v in tokenized_dataset['train'][idx].items():
   print(f'k = {k}')
   print(f'\tv = {v}')

k = id
	v = train_500
k = dialogue
	v = #Person1#: What is your great struck?
#Person2#: I think I'm very good at planning. I manage my time perfectly, so that I can always get things done on time.
#Person1#: Why do you think you be a good fit or this job?
#Person2#: Because I'm very competent in this area.
#Person1#: Can you working under pressure?
#Person2#: Yes, I work well under pressure and find it dimly. However, I believe planning and probable management of my time can reduce tight deadlines.
#Person1#: Are you more a follower or a leader?
#Person2#: I don't try to go ahead of people and lead them, I'd rather cooperate with everybody else and get the job done by working together.
k = summary
	v = #Person1# is interviewing #Person2#. #Person2# is good at planning, works well under pressure, and is a cooperator.
k = topic
	v = interview
k = input_ids
	v = [12198, 1635, 1737, 8, 826, 7478, 5, 1713, 345, 13515, 536, 4663, 10, 363, 19, 39, 248, 10056, 58, 1713, 345, 13515, 357, 4663,

In [16]:
print(tokenizer.decode(tokenized_dataset['train'][idx]['input_ids'], skip_special_tokens=True))
print('-------------------')
print(tokenizer.decode(tokenized_dataset['train'][idx]['labels'], skip_special_tokens=True))

Summarize the following dialogue. #Person1#: What is your great struck? #Person2#: I think I'm very good at planning. I manage my time perfectly, so that I can always get things done on time. #Person1#: Why do you think you be a good fit or this job? #Person2#: Because I'm very competent in this area. #Person1#: Can you working under pressure? #Person2#: Yes, I work well under pressure and find it dimly. However, I believe planning and probable management of my time can reduce tight deadlines. #Person1#: Are you more a follower or a leader? #Person2#: I don't try to go ahead of people and lead them, I'd rather cooperate with everybody else and get the job done by working together. Summary:
-------------------
#Person1# is interviewing #Person2#. #Person2# is good at planning, works well under pressure, and is a cooperator.


In [19]:
# TODO: Remove id, dialogue, summary and topic columns from dataset. We only want input_ids and labels
tokenized_dataset = tokenized_dataset.remove_columns(
   [ 'id', 'dialogue', 'summary', 'topic' ]
)

In [20]:
# TODO: Verify dataset again
idx = 500 
for k, v in tokenized_dataset['train'][idx].items():
   print(f'k = {k}')
   print(f'\tv = {v}')

k = input_ids
	v = [12198, 1635, 1737, 8, 826, 7478, 5, 1713, 345, 13515, 536, 4663, 10, 363, 19, 39, 248, 10056, 58, 1713, 345, 13515, 357, 4663, 10, 27, 317, 27, 31, 51, 182, 207, 44, 1459, 5, 27, 1865, 82, 97, 3923, 6, 78, 24, 27, 54, 373, 129, 378, 612, 30, 97, 5, 1713, 345, 13515, 536, 4663, 10, 1615, 103, 25, 317, 25, 36, 3, 9, 207, 1400, 42, 48, 613, 58, 1713, 345, 13515, 357, 4663, 10, 2070, 27, 31, 51, 182, 11766, 16, 48, 616, 5, 1713, 345, 13515, 536, 4663, 10, 1072, 25, 464, 365, 1666, 58, 1713, 345, 13515, 357, 4663, 10, 2163, 6, 27, 161, 168, 365, 1666, 11, 253, 34, 15688, 120, 5, 611, 6, 27, 857, 1459, 11, 23093, 758, 13, 82, 97, 54, 1428, 4095, 7183, 7, 5, 1713, 345, 13515, 536, 4663, 10, 1521, 25, 72, 3, 9, 1130, 49, 42, 3, 9, 2488, 58, 1713, 345, 13515, 357, 4663, 10, 27, 278, 31, 17, 653, 12, 281, 2177, 13, 151, 11, 991, 135, 6, 27, 31, 26, 1066, 23109, 28, 8611, 1307, 11, 129, 8, 613, 612, 57, 464, 544, 5, 20698, 10, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

### Tune model with pre-processed dataset

We will use [<code>Trainer</code>](https://huggingface.co/docs/transformers/main/en/main_classes/trainer#api-reference%20][%20transformers.Trainer) to train the original model. The training result will be written out. The trainer will be configure with [<code>TrainingArgument</code>](https://huggingface.co/docs/transformers/main/en/main_classes/trainer#transformers.TrainingArguments)

In [21]:
# CUDA information

print('CUDA available: ', torch.cuda.is_available())
if torch.cuda.is_available():
   print('B16 supported: ', torch.cuda.is_bf16_supported())
   torch.cuda.set_device(0)
   print('Current device: ', torch.cuda.current_device())
   print('CUDA device name: ', torch.cuda.get_device_name(0))

CUDA available:  False


In [22]:
print(model)

T5ForConditionalGeneration(
  (shared): Embedding(32128, 768)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 768)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=768, out_features=768, bias=False)
              (k): Linear(in_features=768, out_features=768, bias=False)
              (v): Linear(in_features=768, out_features=768, bias=False)
              (o): Linear(in_features=768, out_features=768, bias=False)
              (relative_attention_bias): Embedding(32, 12)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseGatedActDense(
              (wi_0): Linear(in_features=768, out_features=2048, bias=False)
              (wi_1): Linear(in_features=768, out_features=2048, bias=False)
              (wo):

## Fine tuning the LLM Model with Low-Rank Adaptation (LoRA) / Parameter Efficient Fine Tuning (PEFT)

We will add a LoRA adapter to the LLM (flan-t5-base) and train the adapter. The original LLM will be frozen. The adapter can be combined with the original LLM during inferencing. 

In [29]:
# TODO: Configure LoRA
rank = 16
lora_config = LoraConfig(
   r=rank, # smaller - get better at what you are doing, larger - train to do variation of new things 
   lora_alpha=rank, # controls the merging of LoRA to the model's weights
   target_modules=['q', 'v'],
   lora_dropout=.05,
   bias="none",
   task_type=TaskType.SEQ_2_SEQ_LM
)

In [30]:
# TODO: Add LoRA to the LLM model to be trained
original_model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

# Create a LoRA model with the configuration and a base model
lora_model = get_peft_model(original_model, lora_config)

In [31]:
# TODO: Print number of parameters, compare LoRA to the original model
print_trainable_model_params(lora_model)

'\n   Trainable parameters: 1769472\n   Total parameters: 249347328\n   Percentable of trainable parameters: 0.71%\n   '

In [32]:
# TODO: Train model with LoRA
output_dir = f'peft-dialog-summary-training-{str(int(time.time()))}'


In [33]:
# TODO: Create trainer and train model
lora_train_args = TrainingArguments(
   output_dir=output_dir,
   auto_find_batch_size=True,
   learning_rate=1e-3,
   num_train_epochs=1   
)

In [ ]:
lora_trainer = Trainer(
   model = lora_model,
   args = lora_train_args,
   train_dataset = tokenized_dataset['train'],
   eval_dataset = tokenized_dataset['validation']
)

# Train the model
lora_trainer.train()

# Save the model
#my_peft_model = "my-peft-model"

# Save the model and its corresponding tokenizer.
#lora_trainer.save_pretrained(my_peft_model)
#tokenizer.save_pretrained(my_peft_model)

### Use a trained LoRA model

The training will take a few hours and over many iterations.

For the purpose of this workshop we use a save model [intotheverse/peft-dialogue-summary-checkpoint](https://huggingface.co/intotheverse/peft-dialogue-summary-checkpoint).

In [34]:
#TODO: Load the original model and add the pre-trained LoRA adaptation to the model
peft_dialogue_summary_checkpoint = 'intotheverse/peft-dialogue-summary-checkpoint'

# load the base model
base_model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

# LoRA model
lora_model = PeftModel.from_pretrained(
   base_model, peft_dialogue_summary_checkpoint,
   torch_dtype = torch.bfloat16, 
   is_trainable=False
)

print(print_trainable_model_params(lora_model))


   Trainable parameters: 0
   Total parameters: 251116800
   Percentable of trainable parameters: 0.00%
   


## Evaluate LoRA model

In [35]:
# TODO: Evaluate LoRA model against the original 
original_model = AutoModelForSeq2SeqLM.from_pretrained(model_name)


In [36]:
# Prepare data for evaluation
dialogues = []
summaries = []
orig_model_summaries = []
lora_model_summaries = []
config = GenerationConfig(max_new_tokens=200)

for i in range(5):
   print(f'i = {i}')
   d = dataset['test'][i]['dialogue']
   s = dataset['test'][i]['summary']
   prompt = f"Summarize the following conversation.\n\n{d}\n\nSummary:"
   tokenized_prompt = tokenizer(prompt, return_tensors='pt').input_ids
   orig_resp = original_model.generate(input_ids=tokenized_prompt, generation_config=config)
   orig_resp_text = tokenizer.decode(orig_resp[0], skip_special_tokens=True)
   lora_resp = lora_model.generate(input_ids=tokenized_prompt, generation_config=config)
   lora_resp_text = tokenizer.decode(lora_resp[0], skip_special_tokens=True)

   summaries.append(s)
   orig_model_summaries.append(orig_resp_text)
   lora_model_summaries.append(lora_resp_text)

zipped_summaries = list(zip(summaries, orig_model_summaries, lora_model_summaries))
df = pd.DataFrame(zipped_summaries, columns=['label', 'original_model_summary', 'lora_model_summary'])
df

i = 0
i = 1
i = 2
i = 3
i = 4


,label,original_model_summary,lora_model_summary
0,Ms. Dawson helps #Person1# to write a memo to ...,The memo is to be distributed to all employees...,#Person1# asks Ms. Dawson to take a dictation ...
1,In order to prevent employees from wasting tim...,The memo is to be distributed to all employees...,#Person1# asks Ms. Dawson to take a dictation ...
2,Ms. Dawson takes a dictation for #Person1# abo...,The memo is to be distributed to all employees...,#Person1# asks Ms. Dawson to take a dictation ...
3,#Person2# arrives late because of traffic jam....,The traffic jam at the Carrefour intersection ...,#Person2# got stuck in traffic and #Person1# s...
4,#Person2# decides to follow #Person1#'s sugges...,The traffic jam at the Carrefour intersection ...,#Person2# got stuck in traffic and #Person1# s...


### Evaluate models with ROUGE/Bleu metrics

Recall-Oriented Understudy for Gisting Evaluate ([ROUGE](https://pub.aimind.so/unveiling-the-power-of-rouge-metrics-in-nlp-b6d3f96d3363)) is a set of metrics used to evaluate the quality of machine-generated text, such as summaries and translations. ROUGE metrics compare the generated text to a human-written reference and measure the overlap between the two. 

The metrics range between 0 and 1, with higher scores indicating higher similarity between the baseline and generated text.

In [37]:
# TODO: create ROUGE metrics
rouge = evaluate.load('rouge')

In [38]:
# TODO: Evaluate the original model's result
orig_model_eval_results = rouge.compute(
   references=summaries,
   predictions=orig_model_summaries,
   use_aggregator=True,
   use_stemmer=True
)
print(orig_model_eval_results)

{'rouge1': np.float64(0.17391941391941393), 'rouge2': np.float64(0.03820816864295125), 'rougeL': np.float64(0.13364468864468865), 'rougeLsum': np.float64(0.13364468864468865)}


In [39]:
# TODO: Evaluate with Bleu metrics
lora_model_eval_results = rouge.compute(
   references=summaries,
   predictions=lora_model_summaries,
   use_aggregator=True,
   use_stemmer=True
)
print(lora_model_eval_results)

{'rouge1': np.float64(0.34183605536870687), 'rouge2': np.float64(0.1024924201890494), 'rougeL': np.float64(0.27285117714700724), 'rougeLsum': np.float64(0.2728511771470072)}


In [40]:
print('original model')
print(orig_model_eval_results)

print()

print('lora model')
print(lora_model_eval_results)

original model
{'rouge1': np.float64(0.17391941391941393), 'rouge2': np.float64(0.03820816864295125), 'rougeL': np.float64(0.13364468864468865), 'rougeLsum': np.float64(0.13364468864468865)}

lora model
{'rouge1': np.float64(0.34183605536870687), 'rouge2': np.float64(0.1024924201890494), 'rougeL': np.float64(0.27285117714700724), 'rougeLsum': np.float64(0.2728511771470072)}


In [41]:
bleu = evaluate.load('bleu')

In [44]:
orig_model_eval_bleu_results = bleu.compute(
   references=summaries,
   predictions=orig_model_summaries,
   use_aggregator=True,
   use_stemmer=True
)
print(orig_model_eval_bleu_results)

TypeError: Bleu._compute() got an unexpected keyword argument 'use_aggregator'

In [42]:
lora_model_eval_bleu_results = rouge.compute(
   references=summaries,
   predictions=lora_model_summaries,
   use_aggregator=True,
   use_stemmer=True
)
print(lora_model_eval_bleu_results)

{'rouge1': np.float64(0.34183605536870687), 'rouge2': np.float64(0.1024924201890494), 'rougeL': np.float64(0.27285117714700724), 'rougeLsum': np.float64(0.2728511771470072)}
